In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx


def create_binary_matrix(n, ratio_of_ones):
    total_elements = n * n
    num_ones = int(ratio_of_ones * total_elements)

    # Create a binary matrix with all zeros
    binary_matrix = np.zeros((n, n), dtype=int)

    # Set random positions to ones
    ones_positions = np.random.choice(total_elements, num_ones, replace=False)
    row_indices, col_indices = np.unravel_index(ones_positions, (n, n))
    binary_matrix[row_indices, col_indices] = 1

    return binary_matrix

def graph_network(M):
    # Create a graph from the adjacency matrix
    G = nx.from_numpy_matrix(M)
    # Draw the graph
    pos = nx.circular_layout(G)  # positions for all nodes
    # Get edge weights
    edges = G.edges()
    weights = [G[u][v]['weight'] for u,v in edges]
    # Normalize the weights to [0,1] for mapping to the colorbar
    normalized_weights = [((w - np.min(weights)) / (np.max(weights) - np.min(weights))) for w in weights]
    edges = nx.draw_networkx_edges(G, pos, edge_cmap=plt.cm.coolwarm, edge_color=normalized_weights, edge_vmin=0, edge_vmax=1, width=2)
    # Draw nodes
    nx.draw_networkx_nodes(G, pos)
    # Creating colorbar
    plt.colorbar(edges, label='connection weights')
    plt.show()


# NxN matrix. Each cell represents a node's activity.
N = 50 #number of neurons
sigma = 1 #scaling factor for matrix
C = 0.5 #connectivity matrix
M = np.random.rand(N, N)
M = M / np.sum(M, axis=0) #rescaling so each column summed = 1

C_matrix = create_binary_matrix(N,C)
M = sigma*M #rescaled M matrix based off of sigma

graph_network(M)



NI = 100000 #number of simulation iterations
T = 900 #number of time steps

# Store duration of each experiment and total activity count
durations = np.zeros(NI)
TAC = np.zeros(NI)

for k in range(NI):
    V = np.zeros(N)
    V[0] = 1
    activity_total = 0

    # simulation
    for i in range(T):
        # calculate sum of activity
        activity_total += np.sum(V) 

        # Implement the new propagation criteria for V
        R = np.random.rand(N, N)
        V = np.dot((M > R), V) >= 1

        # check for termination criteria (when the average activity is close to zero)
        if np.sum(V)==0:  # threshold can be adjusted based on your application
            break
       
    durations[k] = i+1
    TAC[k] = activity_total


print("Simulation done!")

# Plotting the results
# Figure 1: Run Duration Histogram
plt.figure(1)
a, b = np.histogram(durations, bins=200000)
plt.loglog(b[:-1], a, 'bo')  # Log-log plot of durations.
plt.title('Run Duration Histogram with -3/2 Line')

# Figure 2: Event Size Histogram
plt.figure(2)
a, b = np.histogram(TAC, bins=200000)
plt.loglog(b[:-1], a, 'ro')  # Log-log plot of event sizes.
plt.title('Event Size Histogram with -3/2 Line')

# Adding -3/2 power-law lines to the plots
for i in range(1, 3):
    plt.figure(i)
    x = np.linspace(1, 100, 100)  # X-axis range for the power-law line.
    k = 1e4  # Constant k for the power-law relationship, adjustable.
    y = k * x**(-3 / 2)  # Calculate y using the power-law relationship.
    plt.loglog(x, y)  # Add the power-law line to the log-log plot.
    plt.grid(True)  # Add a grid for better readability.
    plt.xlabel('log(x)')  # X-axis label.
    plt.ylabel('log(y)')  # Y-axis label.

plt.show()

ModuleNotFoundError: No module named 'networkx'